In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Load pickled data
import pickle
import matplotlib.pyplot as plt

# TODO: Fill this in based on where you saved the training and testing data

training_file = "test.p"
validation_file="valid.p"
testing_file = "train.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']

X_test, y_test = test['features'], test['labels']

In [5]:
import numpy as np
print('train='+str(X_train.shape))
print('test='+str(X_test.shape))
print('validate='+str(X_valid.shape))
# print(X_train.shape)
train_shape=X_train.shape
test_shape=X_test.shape
valid_shape=X_valid.shape
labels_number=len(np.unique(y_train))
labels_number

train=(12630, 32, 32, 3)
test=(34799, 32, 32, 3)
validate=(4410, 32, 32, 3)


43

In [6]:
##Normalisation
X_train=X_train/255.0
X_test=X_test/255.0
X_valid=X_valid/255.0


In [7]:
import tensorflow as tf
from tensorflow import estimator,metrics,sigmoid,acos
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,MaxPool2D,Dropout,Flatten,Conv2D
from tensorflow.keras.activations import relu

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

Using TensorFlow backend.


In [8]:
def build_models(hp):
    model=Sequential([
        Conv2D(
            filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
            kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
            activation='relu',
            input_shape=(32,32,3)
        ),
        Dropout(hp.Choice('d1',values=[1e-2,1e-3])),
        
        Conv2D(filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
               kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
               activation='relu'
        ),
        Dropout(hp.Float('d2',min_value=1e-3,max_value=1e-1)),

        Conv2D(filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
               kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
               activation='relu'
        ),
        Conv2D(filters=hp.Int('conv_3_filter',min_value=32,max_value=128,step=16),kernel_size=(3,3),padding="same", activation="relu"),
        MaxPool2D(pool_size=(2,2),strides=(2,2)),
        Conv2D(filters=hp.Int('conv_4_filter',min_value=32,max_value=256,step=16),kernel_size=(3,3),padding="same", activation="relu"),
        MaxPool2D(pool_size=(3,3),strides=(3,3)),
        
        Dense(units=hp.Int('dense_1_units',
              min_value=32,max_value=128,step=8),
              activation='relu'
        ),
        Dropout(hp.Float('d3',min_value=1e-3,max_value=1e-1)),

        Dense(units=hp.Int('dense_4_units',
              min_value=32,max_value=256,step=8),
              activation='relu'
        ),
        
#         Flatten(),
        Dropout(hp.Float('d4',min_value=1e-3,max_value=1e-1)),
        MaxPool2D(pool_size=(3,3),strides=(3,3)),
        Dense(units=hp.Int('dense_5_units',
              min_value=128,max_value=512,step=32),
              activation='relu'
        ),
        
        Dense(43,activation='softmax')
    ])
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-1,1e-3])),
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    return model

In [9]:
from kerastuner import RandomSearch,HyperParameters #importing HyperParamter and RandomSearch

In [10]:
#now using random serch for this model
tuner_search=RandomSearch(build_models,objective='val_accuracy',max_trials=5,directory='output',project_name='Mnist')

RuntimeError: Model-building function did not return a valid Keras Model instance, found <keras.engine.sequential.Sequential object at 0x0000024045604828>